In [44]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, time
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [45]:
#read excel file 
df = pd.read_excel(r"C:\Users\JoudA\OneDrive\سطح المكتب\fake_medicine_usage.xlsx")
# from string to time form 
df['scheduled_time'] = pd.to_datetime(df['scheduled_time'], format="%H:%M").dt.time
df['taken_time'] = pd.to_datetime(df['taken_time'], format="%H:%M", errors='coerce').dt.time

In [46]:
# calculate the difference between scheduled_time & taken_time
def time_diff(scheduled, taken):
    if pd.isnull(taken):
        return np.nan
    scheduled_dt = datetime.combine(datetime.today(), scheduled)
    taken_dt = datetime.combine(datetime.today(), taken)
    return (taken_dt - scheduled_dt).total_seconds() / 60
# apply on all rows
df['time_diff_minutes'] = df.apply(lambda row: time_diff(row['scheduled_time'], row['taken_time']), axis=1)

In [47]:
# from string to date 
df['date'] = pd.to_datetime(df['date'])
df['week'] = df['date'].dt.isocalendar().week


In [48]:
# statical operations 
# mean for time_diff_minutes / taken_on_time
# std for time_diff_minutes
grouped = df.groupby(['user_id', 'medicine_id', 'week']).agg({
    'time_diff_minutes': ['mean', 'std'],
    'taken_on_time': 'mean'
}).reset_index()

# rename col
grouped.columns = ['user_id', 'medicine_id', 'week', 'avg_diff', 'std_diff', 'on_time_ratio']

In [49]:
# delete all nan rows
model_df = grouped.dropna()  

# determian features & target
X = model_df[['std_diff', 'on_time_ratio']] 
y = model_df['avg_diff'] 

In [50]:
# train the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [51]:
# determain the production & MAE
preds = model.predict(X_test)
mae = mean_absolute_error(y_test, preds)

In [52]:
def suggest_new_time(row):
    base_time_series = df[
        (df['user_id'] == row['user_id']) &
        (df['medicine_id'] == row['medicine_id']) &
        (df['date'].dt.isocalendar().week == row['week'])
    ]['scheduled_time']

    if base_time_series.empty:
        return None

    base_time = base_time_series.iloc[0]

    if isinstance(base_time, pd._libs.tslibs.timestamps.Timestamp):
        base_time_dt = base_time.to_pydatetime()
    elif isinstance(base_time, time):
        base_time_dt = datetime.combine(datetime.today(), base_time)
    elif isinstance(base_time, str):
        base_time_dt = datetime.strptime(base_time, "%H:%M")
    else:
        return None

    pred_diff = model.predict(pd.DataFrame([[row['std_diff'], row['on_time_ratio']]], columns=['std_diff', 'on_time_ratio']))[0]
    new_time = base_time_dt + timedelta(minutes=pred_diff)
    return new_time.strftime("%H:%M")

In [53]:
model_df.loc[:, 'suggested_time'] = model_df.apply(suggest_new_time, axis=1)

C:\Users\JoudA\AppData\Local\Temp\ipykernel_18760\519660041.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df.loc[:, 'suggested_time'] = model_df.apply(suggest_new_time, axis=1)


In [54]:
print(model_df[['user_id', 'medicine_id', 'week', 'suggested_time']].head(100))

    user_id medicine_id  week suggested_time
0      U001         M01    27          11:46
1      U001         M01    28          11:45
2      U001         M01    29          11:44
3      U001         M01    30          11:46
5      U001         M02    27          19:10
..      ...         ...   ...            ...
118    U005         M03    30          11:42
120    U005         M04    27          14:03
121    U005         M04    28          14:01
122    U005         M04    29          14:00
123    U005         M04    30          13:54

[100 rows x 4 columns]


In [55]:
# final 
print(f"Mean Absolute Error: {mae:.2f} minutes")

Mean Absolute Error: 9.04 minutes
